In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

pasta_dados = '../dados/'

df_empreend = pd.read_csv(pasta_dados + 'tratados/simu_carteira_empreendimentos.csv')
df_acidentes = pd.read_csv(pasta_dados + 'tratados/simu_acidentes_de_transportes.csv')

In [2]:
df_empreend.shape

(71248, 121)

In [3]:
df_empreend.columns

Index(['cod_mdr', 'cod_operacao', 'cod_saci', 'cod_convenio_siafi',
       'cod_ag_operador', 'origem', 'dsc_concedente', 'dsc_fonte',
       'dsc_unidade', 'cod_proposta',
       ...
       'aux_emp_viaria', 'aux_emp_aero', 'aux_emp_revitalizacao',
       'aux_emp_infraestrutura', 'aux_emp_pac2etapa',
       'aux_emp_faixa_exclusiva', 'aux_emp_rapido', 'aux_emp_sarjeta',
       'aux_emp_nan', 'new_empreend'],
      dtype='object', length=121)

## Agrupamento dos empreendimentos por município e ano

In [4]:
df_empreend[['vlr_investimento', 'pop_beneficiada']].isna().sum()

vlr_investimento    1036
pop_beneficiada        0
dtype: int64

In [5]:
df_empreend[df_empreend['vlr_investimento'].isnull() &df_empreend['Código IBGE'].notnull() ]

,cod_mdr,cod_operacao,cod_saci,cod_convenio_siafi,cod_ag_operador,origem,dsc_concedente,dsc_fonte,dsc_unidade,cod_proposta,dsc_situacao_contrato_mdr,bln_carteira_mdr,bln_carteira_mdr_ativo,dsc_situacao_objeto_mdr,dsc_paralisada_mdr,bln_ativo,bln_ativo_boolean,cod_id_fonte,cod_id_pac,programa,acao,uf_SIGLA_UF,mun_MUNNOMEX,Código IBGE,empreendimento,vlr_repasse_financiamento,vlr_contrapartida,vlr_empenhado,vlr_desembolsado,vlr_desbloqueado,vlr_desembolsado_19,vlr_desembolsado_20,vlr_desembolsado_21,situacao_obra,situacao_obra_base_validacao,situacao_contrato,prc_fisico,dte_assinatura,dte_inicio_obra,dte_fim_obra,dte_ult_desembolso,dte_ult_desbloqueio,dte_ult_ref_bm,vlr_ult_desembolso,vlr_ult_repasse_desbloqueado,vlr_ult_contrapartida_desbloqueado,agente_financeiro,agente_financeiro_saci,dias_paralisados,situacao_obra_base_validacao_secex,situacao_obra_mdr,pop_beneficiada,emp_gerado,vlr_investimento,vlr_ult_investimento_desbloqueado,vlr_repasse_financiamento_num,vlr_contrapartida_num,vlr_investimento_num,vlr_empenhado_num,vlr_desembolsado_num,vlr_desbloqueado_num,vlr_desembolsado_19_num,vlr_desembolsado_20_num,vlr_desembolsado_21_num,vlr_ult_desembolso_num,vlr_ult_repasse_desbloqueado_num,vlr_ult_contrapartida_desbloqueado_num,vlr_ult_investimento_desbloqueado_num,ano_assinatura,ano_inicio_obra,ano_fim_obra,ano_ult_desembolso,ano_ult_desbloqueio,ano_ult_ref_bm,ano,Populacao,Região,uf_REGIAO,Município,mun_coordenadas,mun_MUNNOME,mun_AMAZONIA,mun_FRONTEIRA,mun_CAPITAL,uf_CODIGO_UF,mun_LATITUDE,mun_LONGITUDE,mun_ALTITUDE,mun_AREA,mun_codigo_adotado,uf_NOME_UF,aux_emp_implantacao,aux_emp_pavimentacao,aux_emp_refrota,aux_emp_sinalizacao,aux_emp_calcamento,aux_emp_ciclovia,aux_emp_corredor,aux_emp_drenagem,aux_emp_ferrovia,aux_emp_transporte,aux_emp_brt,aux_emp_vlt,aux_emp_qualificacao,aux_emp_mobilidade_urbana,aux_emp_aquisicao,aux_emp_polongamento,aux_emp_ampliacao,aux_emp_recapeamento,aux_emp_linha,aux_emp_metro,aux_emp_viaria,aux_emp_aero,aux_emp_revitalizacao,aux_emp_infraestrutura,aux_emp_pac2etapa,aux_emp_faixa_exclusiva,aux_emp_rapido,aux_emp_sarjeta,aux_emp_nan,new_empreend


In [6]:
import numpy as np

def agrupar_base_empreendimentos(df_empreend, ano=True):

    df = df_empreend.copy()

    aux_emp_columns = df.filter(like='aux_emp').columns

    agg_dict = {
        'vlr_investimento': 'sum',
        'Populacao': 'mean',
        'pop_beneficiada': 'sum',
        'Código IBGE': 'size'
    }

    # Para calcular o total de investimentos em cada categoria de empreendimentos, vamos primeiro criar uma coluna para cada categoria
    # que é igual ao investimento daquele empreendimento se coluna não nula e igual a zero se coluna não zero
    # Em seguida, quando agruparmos as colunas vamos somar o valor de investimento em cada categoria
    for column in aux_emp_columns:
        agg_dict[column] = lambda x: x.notnull().sum()

        new_column_name = column + '_vlr_investimento'
        
        # Calcula a soma dos valores da coluna "vlr_investimento" onde a coluna aux_emp correspondente não é nula
        df[new_column_name] = df.apply(lambda row: row['vlr_investimento'] if pd.notnull(row[column]) else 0, axis=1)
        
        agg_dict[new_column_name] = 'sum'

    # Cria colunas dummy para cada valor distinto na coluna 'programa' - Faremos o mesmo que fizemos com as categorias de empreendimentos
    programa_dummies = pd.get_dummies(df['programa'], prefix='programa')

    df = pd.concat([df, programa_dummies], axis=1)

    for column in programa_dummies.columns:
        agg_dict[column] = 'sum'
        
        new_column_name = column + '_vlr_investimento'
        
        # Calcula a soma dos valores da coluna "vlr_investimento" onde a coluna 'programa' corresponde a cada valor distinto
        df[new_column_name] = df.apply(lambda row: row['vlr_investimento'] if row['programa'] == column.replace('programa_', '') else 0, axis=1)
        
        agg_dict[new_column_name] = 'sum'
    
    if ano:
        colunas_agrupar = ['Código IBGE', 'ano']
    else:
        colunas_agrupar = ['Código IBGE']

    df_agrupado_municipio_empreend = df.groupby(colunas_agrupar).agg(agg_dict)

    df_agrupado_municipio_empreend.rename(columns={'Código IBGE': 'num_total_empreendimentos' }, inplace=True)

    # Também vamos calcular o valor de investimento e a população beneficiada per capita
    df_agrupado_municipio_empreend['vlr_investimento_per_capita'] = df_agrupado_municipio_empreend['vlr_investimento'] / df_agrupado_municipio_empreend['Populacao']
    df_agrupado_municipio_empreend['pop_beneficiada_per_capita'] = df_agrupado_municipio_empreend['pop_beneficiada'] / df_agrupado_municipio_empreend['Populacao']

    return df_agrupado_municipio_empreend

In [7]:
df_agrupado_municipio_empreend = agrupar_base_empreendimentos(df_empreend)

In [8]:
# codigo = 1100023.0
# ano = 2005.0

codigo = 2927408.0
ano = 2021.0	

In [9]:
pd.DataFrame(df_agrupado_municipio_empreend.loc[(codigo, ano)]).transpose()[['aux_emp_implantacao_vlr_investimento', 'aux_emp_pavimentacao_vlr_investimento',
                                                                             'programa_EMENDAS', 'programa_EMENDAS_vlr_investimento',
                                                                             'programa_SETOR PRIVADO', 'programa_SETOR PRIVADO_vlr_investimento',
                                                                             ]]

,,aux_emp_implantacao_vlr_investimento,aux_emp_pavimentacao_vlr_investimento,programa_EMENDAS,programa_EMENDAS_vlr_investimento,programa_SETOR PRIVADO,programa_SETOR PRIVADO_vlr_investimento
2927408.0,2021.0,0.0,2057270.0,4.0,15512386.19,10.0,44847000.0


In [10]:
df_empreend[(df_empreend['Código IBGE'] == codigo) & (df_empreend['ano'] == ano)][['vlr_investimento', 'aux_emp_pavimentacao', 'aux_emp_implantacao', 'programa']]

,vlr_investimento,aux_emp_pavimentacao,aux_emp_implantacao,programa
66271,1377270.00,PAVIMENTACAO,NaN,EMENDAS
66280,680000.00,PAVIMENTACAO,NaN,EMENDAS
67034,8601596.22,NaN,NaN,EMENDAS
69328,4853519.97,NaN,NaN,EMENDAS
71238,5525000.00,NaN,NaN,SETOR PRIVADO
71239,5800000.00,NaN,NaN,SETOR PRIVADO
71240,5967000.00,NaN,NaN,SETOR PRIVADO
71241,6264000.00,NaN,NaN,SETOR PRIVADO
71242,5568000.00,NaN,NaN,SETOR PRIVADO
71243,2873000.00,NaN,NaN,SETOR PRIVADO


In [11]:
1377270.00	+ 680000.00

2057270.0

In [12]:
df_empreend[(df_empreend['Código IBGE'] == codigo) & (df_empreend['ano'] == ano) & (df_empreend['programa'] == 'SETOR PRIVADO')]['vlr_investimento'].sum()

44847000.0

In [13]:
df_empreend[(df_empreend['Código IBGE'] == codigo) & (df_empreend['ano'] == ano) & (df_empreend['programa'] == 'SETOR PRIVADO')][['vlr_investimento', 'aux_emp_pavimentacao', 'aux_emp_implantacao', 'programa']]

,vlr_investimento,aux_emp_pavimentacao,aux_emp_implantacao,programa
71238,5525000.0,NaN,NaN,SETOR PRIVADO
71239,5800000.0,NaN,NaN,SETOR PRIVADO
71240,5967000.0,NaN,NaN,SETOR PRIVADO
71241,6264000.0,NaN,NaN,SETOR PRIVADO
71242,5568000.0,NaN,NaN,SETOR PRIVADO
71243,2873000.0,NaN,NaN,SETOR PRIVADO
71244,3016000.0,NaN,NaN,SETOR PRIVADO
71245,5304000.0,NaN,NaN,SETOR PRIVADO
71246,2210000.0,NaN,NaN,SETOR PRIVADO
71247,2320000.0,NaN,NaN,SETOR PRIVADO


In [14]:
for i in df_agrupado_municipio_empreend.columns:
    print(i)

vlr_investimento
Populacao
pop_beneficiada
num_total_empreendimentos
aux_emp_implantacao
aux_emp_implantacao_vlr_investimento
aux_emp_pavimentacao
aux_emp_pavimentacao_vlr_investimento
aux_emp_refrota
aux_emp_refrota_vlr_investimento
aux_emp_sinalizacao
aux_emp_sinalizacao_vlr_investimento
aux_emp_calcamento
aux_emp_calcamento_vlr_investimento
aux_emp_ciclovia
aux_emp_ciclovia_vlr_investimento
aux_emp_corredor
aux_emp_corredor_vlr_investimento
aux_emp_drenagem
aux_emp_drenagem_vlr_investimento
aux_emp_ferrovia
aux_emp_ferrovia_vlr_investimento
aux_emp_transporte
aux_emp_transporte_vlr_investimento
aux_emp_brt
aux_emp_brt_vlr_investimento
aux_emp_vlt
aux_emp_vlt_vlr_investimento
aux_emp_qualificacao
aux_emp_qualificacao_vlr_investimento
aux_emp_mobilidade_urbana
aux_emp_mobilidade_urbana_vlr_investimento
aux_emp_aquisicao
aux_emp_aquisicao_vlr_investimento
aux_emp_polongamento
aux_emp_polongamento_vlr_investimento
aux_emp_ampliacao
aux_emp_ampliacao_vlr_investimento
aux_emp_recapeamento

In [15]:
df_agrupado_municipio_empreend.shape

(45979, 76)

## Combinação da base de empreendimentos com a de acidentes

In [16]:
df_acidentes

,Código IBGE,ano,Região,Pedestre_mortes,Ciclista_mortes,Motociclista_mortes,Ocup_triciclo_motor_mortes,Ocup_automovel_mortes,Ocup_caminhonete_mortes,Ocup_veic_transp_pesado_mortes,Ocup_onibus_mortes,Outros_mortes,Populacao_acidentes,total_mortes,taxa_mun_mortes,taxa_mun_pedestre_mortes,taxa_mun_ciclista_mortes,taxa_mun_motociclista_mortes,taxa_mun_automovel_mortes,taxa_mun_veiculo_pesado_mortes,taxa_mun_onibus_mortes,taxa_mun_outros_mortes,Pedestre_feridos,Ciclista_feridos,Motociclista_feridos,Ocup_triciclo_motor_feridos,Ocup_automovel_feridos,Ocup_caminhonete_feridos,Ocup_veic_transp_pesado_feridos,Ocup_onibus_feridos,Outros_feridos,total_feridos,taxa_mun_feridos,taxa_mun_pedestre_feridos,taxa_mun_ciclista_feridos,taxa_mun_motociclista_feridos,taxa_mun_automovel_feridos,taxa_mun_veiculo_pesado_feridos,taxa_mun_onibus_feridos,taxa_mun_outros_feridos,Populacao_populacao,taxa_mun_triciclo_mortes,taxa_mun_caminhonete_mortes,taxa_mun_triciclo_feridos,taxa_mun_caminhonete_feridos
0,1100015,2010,N,2,0,5,0,0,0,0,0,0,24392,7.0,0.028698,0.008199,0.000000,0.020499,0.000000,0.000000,0.000000,0.000000,0,0,13,0,0,0,0,0,0,13.0,0.053296,0.000000,0.000000,0.053296,0.000000,0.000000,0.000000,0.000000,24392.0,0.0,0.000000,0.000000,0.000000
1,1100023,2010,N,0,1,2,0,0,0,0,1,29,90353,33.0,0.036523,0.000000,0.001107,0.002214,0.000000,0.000000,0.001107,0.032096,0,1,37,0,1,1,0,0,3,43.0,0.047591,0.000000,0.001107,0.040950,0.001107,0.000000,0.000000,0.003320,90353.0,0.0,0.000000,0.000000,0.001107
2,1100031,2010,N,0,0,0,0,0,0,0,0,0,6313,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6313.0,0.0,0.000000,0.000000,0.000000
3,1100049,2010,N,2,1,17,0,4,0,0,0,2,78574,26.0,0.033090,0.002545,0.001273,0.021636,0.005091,0.000000,0.000000,0.002545,50,1,5,0,0,0,0,0,1,57.0,0.072543,0.063634,0.001273,0.006363,0.000000,0.000000,0.000000,0.001273,78574.0,0.0,0.000000,0.000000,0.000000
4,1100056,2010,N,3,0,1,0,0,0,0,0,0,17029,4.0,0.023489,0.017617,0.000000,0.005872,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0,0,0,0,0,1.0,0.005872,0.000000,0.005872,0.000000,0.000000,0.000000,0.000000,0.000000,17029.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55695,5222005,2019,CO,1,0,1,0,0,0,0,0,2,13863,4.0,0.028854,0.007213,0.000000,0.007213,0.000000,0.000000,0.000000,0.014427,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13863.0,0.0,0.000000,0.000000,0.000000
55696,5222054,2019,CO,0,0,0,0,2,0,0,0,2,8743,4.0,0.045751,0.000000,0.000000,0.000000,0.022875,0.000000,0.000000,0.022875,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8743.0,0.0,0.000000,0.000000,0.000000
55697,5222203,2019,CO,0,0,1,0,6,0,1,0,0,6171,8.0,0.129639,0.000000,0.000000,0.016205,0.097229,0.016205,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6171.0,0.0,0.000000,0.000000,0.000000
55698,5222302,2019,CO,0,0,0,0,1,0,0,0,2,5821,3.0,0.051538,0.000000,0.000000,0.000000,0.017179,0.000000,0.000000,0.034358,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5821.0,0.0,0.000000,0.000000,0.000000


Sabemos que df_acidentes é único por conjunto Código IBGE e ano

In [17]:
df_agrupado_municipio_empreend.shape

(45979, 76)

In [18]:
df_acidentes.shape

(55700, 45)

In [19]:
list(set(df_agrupado_municipio_empreend.columns).intersection(set(df_acidentes.columns)))

[]

In [20]:
df_merge_empreendimentos_acidentes = pd.merge(df_agrupado_municipio_empreend, df_acidentes, on=['Código IBGE', 'ano'], how='outer')
df_merge_empreendimentos_acidentes.shape

(80979, 121)

Muitas linhas surgiram, pois a interseção de conjuntos Código IBGE e ano que possuem algum empreendimento e ao mesmo tempo dados de mortes e feridos é só de 20700

In [21]:
pd.merge(df_agrupado_municipio_empreend, df_acidentes, on=['Código IBGE', 'ano'], how='inner').shape

(20700, 121)

In [22]:
unique_rows = df_merge_empreendimentos_acidentes.duplicated(subset=['Código IBGE', 'ano'], keep=False)

if unique_rows.sum() == 0:
    print("O conjunto dos valores das colunas 'Código IBGE' e 'ano' é único.")
else:
    print("O conjunto dos valores das colunas 'Código IBGE' e 'ano' não é único.")


O conjunto dos valores das colunas 'Código IBGE' e 'ano' é único.


In [23]:
df_merge_empreendimentos_acidentes

,Código IBGE,ano,vlr_investimento,Populacao,pop_beneficiada,num_total_empreendimentos,aux_emp_implantacao,aux_emp_implantacao_vlr_investimento,aux_emp_pavimentacao,aux_emp_pavimentacao_vlr_investimento,aux_emp_refrota,aux_emp_refrota_vlr_investimento,aux_emp_sinalizacao,aux_emp_sinalizacao_vlr_investimento,aux_emp_calcamento,aux_emp_calcamento_vlr_investimento,aux_emp_ciclovia,aux_emp_ciclovia_vlr_investimento,aux_emp_corredor,aux_emp_corredor_vlr_investimento,aux_emp_drenagem,aux_emp_drenagem_vlr_investimento,aux_emp_ferrovia,aux_emp_ferrovia_vlr_investimento,aux_emp_transporte,aux_emp_transporte_vlr_investimento,aux_emp_brt,aux_emp_brt_vlr_investimento,aux_emp_vlt,aux_emp_vlt_vlr_investimento,aux_emp_qualificacao,aux_emp_qualificacao_vlr_investimento,aux_emp_mobilidade_urbana,aux_emp_mobilidade_urbana_vlr_investimento,aux_emp_aquisicao,aux_emp_aquisicao_vlr_investimento,aux_emp_polongamento,aux_emp_polongamento_vlr_investimento,aux_emp_ampliacao,aux_emp_ampliacao_vlr_investimento,aux_emp_recapeamento,aux_emp_recapeamento_vlr_investimento,aux_emp_linha,aux_emp_linha_vlr_investimento,aux_emp_metro,aux_emp_metro_vlr_investimento,aux_emp_viaria,aux_emp_viaria_vlr_investimento,aux_emp_aero,aux_emp_aero_vlr_investimento,aux_emp_revitalizacao,aux_emp_revitalizacao_vlr_investimento,aux_emp_infraestrutura,aux_emp_infraestrutura_vlr_investimento,aux_emp_pac2etapa,aux_emp_pac2etapa_vlr_investimento,aux_emp_faixa_exclusiva,aux_emp_faixa_exclusiva_vlr_investimento,aux_emp_rapido,aux_emp_rapido_vlr_investimento,aux_emp_sarjeta,aux_emp_sarjeta_vlr_investimento,aux_emp_nan,aux_emp_nan_vlr_investimento,programa_AVANÇAR PÚBLICO,programa_AVANÇAR PÚBLICO_vlr_investimento,programa_EMENDAS,programa_EMENDAS_vlr_investimento,programa_PAC FIN,programa_PAC FIN_vlr_investimento,programa_PAC OGU,programa_PAC OGU_vlr_investimento,programa_PRÓ COMUNIDADE,programa_PRÓ COMUNIDADE_vlr_investimento,programa_SETOR PRIVADO,programa_SETOR PRIVADO_vlr_investimento,vlr_investimento_per_capita,pop_beneficiada_per_capita,Região,Pedestre_mortes,Ciclista_mortes,Motociclista_mortes,Ocup_triciclo_motor_mortes,Ocup_automovel_mortes,Ocup_caminhonete_mortes,Ocup_veic_transp_pesado_mortes,Ocup_onibus_mortes,Outros_mortes,Populacao_acidentes,total_mortes,taxa_mun_mortes,taxa_mun_pedestre_mortes,taxa_mun_ciclista_mortes,taxa_mun_motociclista_mortes,taxa_mun_automovel_mortes,taxa_mun_veiculo_pesado_mortes,taxa_mun_onibus_mortes,taxa_mun_outros_mortes,Pedestre_feridos,Ciclista_feridos,Motociclista_feridos,Ocup_triciclo_motor_feridos,Ocup_automovel_feridos,Ocup_caminhonete_feridos,Ocup_veic_transp_pesado_feridos,Ocup_onibus_feridos,Outros_feridos,total_feridos,taxa_mun_feridos,taxa_mun_pedestre_feridos,taxa_mun_ciclista_feridos,taxa_mun_motociclista_feridos,taxa_mun_automovel_feridos,taxa_mun_veiculo_pesado_feridos,taxa_mun_onibus_feridos,taxa_mun_outros_feridos,Populacao_populacao,taxa_mun_triciclo_mortes,taxa_mun_caminhonete_mortes,taxa_mun_triciclo_feridos,taxa_mun_caminhonete_feridos
0,1100015.0,2011.0,1530000.00,24228.0,1841.0,2.0,0.0,0.0,2.0,1530000.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1530000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1530000.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63.150074,0.075986,N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24228.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24228.0,0.0,0.000000,0.0,0.0
1,1100015.0,2021.0,1000000.00,22516.0,1193.0,1.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1000000.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.412862,0.052985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [24]:
df_merge_empreendimentos_acidentes.to_csv('../dados/tratados/agrupamento_empreend_acidentes_por_municipio_e_ano.csv', index=False)

# Agrupar tudo do município, juntando os anos

In [25]:
df_agrupado_municipio_empreend_anos_juntos = agrupar_base_empreendimentos(df_empreend, ano=False)
# df_agrupado_municipio_empreend_anos_juntos = df_agrupado_municipio_empreend_anos_juntos.reset_index()

In [26]:
df_agrupado_municipio_empreend_anos_juntos

,vlr_investimento,Populacao,pop_beneficiada,num_total_empreendimentos,aux_emp_implantacao,aux_emp_implantacao_vlr_investimento,aux_emp_pavimentacao,aux_emp_pavimentacao_vlr_investimento,aux_emp_refrota,aux_emp_refrota_vlr_investimento,aux_emp_sinalizacao,aux_emp_sinalizacao_vlr_investimento,aux_emp_calcamento,aux_emp_calcamento_vlr_investimento,aux_emp_ciclovia,aux_emp_ciclovia_vlr_investimento,aux_emp_corredor,aux_emp_corredor_vlr_investimento,aux_emp_drenagem,aux_emp_drenagem_vlr_investimento,aux_emp_ferrovia,aux_emp_ferrovia_vlr_investimento,aux_emp_transporte,aux_emp_transporte_vlr_investimento,aux_emp_brt,aux_emp_brt_vlr_investimento,aux_emp_vlt,aux_emp_vlt_vlr_investimento,aux_emp_qualificacao,aux_emp_qualificacao_vlr_investimento,aux_emp_mobilidade_urbana,aux_emp_mobilidade_urbana_vlr_investimento,aux_emp_aquisicao,aux_emp_aquisicao_vlr_investimento,aux_emp_polongamento,aux_emp_polongamento_vlr_investimento,aux_emp_ampliacao,aux_emp_ampliacao_vlr_investimento,aux_emp_recapeamento,aux_emp_recapeamento_vlr_investimento,aux_emp_linha,aux_emp_linha_vlr_investimento,aux_emp_metro,aux_emp_metro_vlr_investimento,aux_emp_viaria,aux_emp_viaria_vlr_investimento,aux_emp_aero,aux_emp_aero_vlr_investimento,aux_emp_revitalizacao,aux_emp_revitalizacao_vlr_investimento,aux_emp_infraestrutura,aux_emp_infraestrutura_vlr_investimento,aux_emp_pac2etapa,aux_emp_pac2etapa_vlr_investimento,aux_emp_faixa_exclusiva,aux_emp_faixa_exclusiva_vlr_investimento,aux_emp_rapido,aux_emp_rapido_vlr_investimento,aux_emp_sarjeta,aux_emp_sarjeta_vlr_investimento,aux_emp_nan,aux_emp_nan_vlr_investimento,programa_AVANÇAR PÚBLICO,programa_AVANÇAR PÚBLICO_vlr_investimento,programa_EMENDAS,programa_EMENDAS_vlr_investimento,programa_PAC FIN,programa_PAC FIN_vlr_investimento,programa_PAC OGU,programa_PAC OGU_vlr_investimento,programa_PRÓ COMUNIDADE,programa_PRÓ COMUNIDADE_vlr_investimento,programa_SETOR PRIVADO,programa_SETOR PRIVADO_vlr_investimento,vlr_investimento_per_capita,pop_beneficiada_per_capita
Código IBGE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1100015.0,2.530000e+06,2.365733e+04,3034.0,3,0,0.000000e+00,2,1.530000e+06,0,0.0,0,0.00,0,0.0,0,0.00,0,0.000000e+00,0,0.000000e+00,0,0.0,0,0.000000e+00,0,0.000000e+00,0,0.000000e+00,0,0.000000e+00,3,2530000.00,0,0.0,0,0.0,0,0.000000e+00,0,0.00,0,0.000000e+00,0,0.000000e+00,0,0.0,0,0.000000e+00,0,0.0,0,0.00,0,0.0,0,0.0,0,0.0,0,0.00,0,0.00,0,0.0,3,2.530000e+06,0,0.000000e+00,0,0.000000e+00,0,0.0,0,0.0,106.943583,0.128248
1100023.0,5.172818e+07,8.014420e+04,60220.0,15,3,2.938773e+06,10,4.591718e+07,0,0.0,0,0.00,0,0.0,1,1251042.30,0,0.000000e+00,6,7.144334e+06,0,0.0,1,3.684211e+07,0,0.000000e+00,0,0.000000e+00,1,3.684211e+07,5,40480739.47,0,0.0,0,0.0,0,0.000000e+00,0,0.00,0,0.000000e+00,0,0.000000e+00,0,0.0,0,0.000000e+00,0,0.0,3,3225839.08,0,0.0,0,0.0,0,0.0,0,0.00,0,0.00,0,0.0,14,1.488608e+07,1,3.684211e+07,0,0.000000e+00,0,0.0,0,0.0,645.438850,0.751396
1100031.0,3.497805e+05,7.242000e+03,410.0,2,0,0.000000e+00,2,3.497805e+05,0,0.0,0,0.00,0,0.0,0,0.00,0,0.000000e+00,1,2.658500e+05,0,0.0,0,0.000000e+00,0,0.000000e+00,0,0.000000e+00,0,0.000000e+00,0,0.00,0,0.0,0,0.0,0,0.000000e+00,0,0.00,0,0.000000e+00,0,0.000000e+00,0,0.0,0,0.000000e+00,0,0.0,1,83930.51,0,0.0,0,0.0,0,0.0,0,0.00,0,0.00,0,0.0,2,3.497805e+05,0,0.000000e+00,0,0.000000e+00,0,0.0,0,0.0,48.298883,0.056614
1100049.0,8.067858e+07,7.571084e+04,88575.0,19,1,2.045840e+05,14,6.605357e+07,0,0.0,0,0.00,0,0.0,0,0.00,0,0.000000e+00,4,2.278389e+07,0,0.0,2,3.786211e+07,0,0.000000e+00,0,0.000000e+00,3,4.971193e+07,6,18404288.66,0,0.0,0,0.0,0,0.000000e+00,1,530600.00,0,0.000000e+00,1,2.294597e+06,0,0.0,0,0.000000e+00,0,0.0,2,1224584.00,0,0.0,0,0.0,0,0.0,0,0.00,0,0.00,0,0.0,17,3.479553e+07,2,4.588305e+07,0,0.000000e+00,0,0.0,0,0.0,1065.614652,1.169912
1100056.0,9.547742e+06,1.704967e+04,10741.0,9,1,1.496515e+05,7,8.965754e+06,0,0.0,0,0.00,0,0.0,0,0.00,0,0.000000e+00,3,3.787707e+06,0,0.0,0,0.000000e+00,0,0.000000e+00,0,0.000000e+00,0,0.0

In [27]:
# codigo = 1100023.0
# ano = 2005.0

codigo = 1100015.0	

In [28]:
df_agrupado_municipio_empreend_anos_juntos

,vlr_investimento,Populacao,pop_beneficiada,num_total_empreendimentos,aux_emp_implantacao,aux_emp_implantacao_vlr_investimento,aux_emp_pavimentacao,aux_emp_pavimentacao_vlr_investimento,aux_emp_refrota,aux_emp_refrota_vlr_investimento,aux_emp_sinalizacao,aux_emp_sinalizacao_vlr_investimento,aux_emp_calcamento,aux_emp_calcamento_vlr_investimento,aux_emp_ciclovia,aux_emp_ciclovia_vlr_investimento,aux_emp_corredor,aux_emp_corredor_vlr_investimento,aux_emp_drenagem,aux_emp_drenagem_vlr_investimento,aux_emp_ferrovia,aux_emp_ferrovia_vlr_investimento,aux_emp_transporte,aux_emp_transporte_vlr_investimento,aux_emp_brt,aux_emp_brt_vlr_investimento,aux_emp_vlt,aux_emp_vlt_vlr_investimento,aux_emp_qualificacao,aux_emp_qualificacao_vlr_investimento,aux_emp_mobilidade_urbana,aux_emp_mobilidade_urbana_vlr_investimento,aux_emp_aquisicao,aux_emp_aquisicao_vlr_investimento,aux_emp_polongamento,aux_emp_polongamento_vlr_investimento,aux_emp_ampliacao,aux_emp_ampliacao_vlr_investimento,aux_emp_recapeamento,aux_emp_recapeamento_vlr_investimento,aux_emp_linha,aux_emp_linha_vlr_investimento,aux_emp_metro,aux_emp_metro_vlr_investimento,aux_emp_viaria,aux_emp_viaria_vlr_investimento,aux_emp_aero,aux_emp_aero_vlr_investimento,aux_emp_revitalizacao,aux_emp_revitalizacao_vlr_investimento,aux_emp_infraestrutura,aux_emp_infraestrutura_vlr_investimento,aux_emp_pac2etapa,aux_emp_pac2etapa_vlr_investimento,aux_emp_faixa_exclusiva,aux_emp_faixa_exclusiva_vlr_investimento,aux_emp_rapido,aux_emp_rapido_vlr_investimento,aux_emp_sarjeta,aux_emp_sarjeta_vlr_investimento,aux_emp_nan,aux_emp_nan_vlr_investimento,programa_AVANÇAR PÚBLICO,programa_AVANÇAR PÚBLICO_vlr_investimento,programa_EMENDAS,programa_EMENDAS_vlr_investimento,programa_PAC FIN,programa_PAC FIN_vlr_investimento,programa_PAC OGU,programa_PAC OGU_vlr_investimento,programa_PRÓ COMUNIDADE,programa_PRÓ COMUNIDADE_vlr_investimento,programa_SETOR PRIVADO,programa_SETOR PRIVADO_vlr_investimento,vlr_investimento_per_capita,pop_beneficiada_per_capita
Código IBGE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1100015.0,2.530000e+06,2.365733e+04,3034.0,3,0,0.000000e+00,2,1.530000e+06,0,0.0,0,0.00,0,0.0,0,0.00,0,0.000000e+00,0,0.000000e+00,0,0.0,0,0.000000e+00,0,0.000000e+00,0,0.000000e+00,0,0.000000e+00,3,2530000.00,0,0.0,0,0.0,0,0.000000e+00,0,0.00,0,0.000000e+00,0,0.000000e+00,0,0.0,0,0.000000e+00,0,0.0,0,0.00,0,0.0,0,0.0,0,0.0,0,0.00,0,0.00,0,0.0,3,2.530000e+06,0,0.000000e+00,0,0.000000e+00,0,0.0,0,0.0,106.943583,0.128248
1100023.0,5.172818e+07,8.014420e+04,60220.0,15,3,2.938773e+06,10,4.591718e+07,0,0.0,0,0.00,0,0.0,1,1251042.30,0,0.000000e+00,6,7.144334e+06,0,0.0,1,3.684211e+07,0,0.000000e+00,0,0.000000e+00,1,3.684211e+07,5,40480739.47,0,0.0,0,0.0,0,0.000000e+00,0,0.00,0,0.000000e+00,0,0.000000e+00,0,0.0,0,0.000000e+00,0,0.0,3,3225839.08,0,0.0,0,0.0,0,0.0,0,0.00,0,0.00,0,0.0,14,1.488608e+07,1,3.684211e+07,0,0.000000e+00,0,0.0,0,0.0,645.438850,0.751396
1100031.0,3.497805e+05,7.242000e+03,410.0,2,0,0.000000e+00,2,3.497805e+05,0,0.0,0,0.00,0,0.0,0,0.00,0,0.000000e+00,1,2.658500e+05,0,0.0,0,0.000000e+00,0,0.000000e+00,0,0.000000e+00,0,0.000000e+00,0,0.00,0,0.0,0,0.0,0,0.000000e+00,0,0.00,0,0.000000e+00,0,0.000000e+00,0,0.0,0,0.000000e+00,0,0.0,1,83930.51,0,0.0,0,0.0,0,0.0,0,0.00,0,0.00,0,0.0,2,3.497805e+05,0,0.000000e+00,0,0.000000e+00,0,0.0,0,0.0,48.298883,0.056614
1100049.0,8.067858e+07,7.571084e+04,88575.0,19,1,2.045840e+05,14,6.605357e+07,0,0.0,0,0.00,0,0.0,0,0.00,0,0.000000e+00,4,2.278389e+07,0,0.0,2,3.786211e+07,0,0.000000e+00,0,0.000000e+00,3,4.971193e+07,6,18404288.66,0,0.0,0,0.0,0,0.000000e+00,1,530600.00,0,0.000000e+00,1,2.294597e+06,0,0.0,0,0.000000e+00,0,0.0,2,1224584.00,0,0.0,0,0.0,0,0.0,0,0.00,0,0.00,0,0.0,17,3.479553e+07,2,4.588305e+07,0,0.000000e+00,0,0.0,0,0.0,1065.614652,1.169912
1100056.0,9.547742e+06,1.704967e+04,10741.0,9,1,1.496515e+05,7,8.965754e+06,0,0.0,0,0.00,0,0.0,0,0.00,0,0.000000e+00,3,3.787707e+06,0,0.0,0,0.000000e+00,0,0.000000e+00,0,0.000000e+00,0,0.0

In [29]:
codigo in df_agrupado_municipio_empreend_anos_juntos.index

True

In [30]:
pd.DataFrame(df_agrupado_municipio_empreend_anos_juntos.loc[codigo]).transpose()[['aux_emp_implantacao_vlr_investimento', 'aux_emp_pavimentacao_vlr_investimento',
                                                                             'programa_EMENDAS', 'programa_EMENDAS_vlr_investimento',
                                                                             'programa_SETOR PRIVADO', 'programa_SETOR PRIVADO_vlr_investimento',
                                                                             ]]

,aux_emp_implantacao_vlr_investimento,aux_emp_pavimentacao_vlr_investimento,programa_EMENDAS,programa_EMENDAS_vlr_investimento,programa_SETOR PRIVADO,programa_SETOR PRIVADO_vlr_investimento
1100015.0,0.0,1530000.0,3.0,2530000.0,0.0,0.0


In [31]:
df_empreend[(df_empreend['Código IBGE'] == codigo)][['vlr_investimento', 'aux_emp_pavimentacao', 'aux_emp_implantacao', 'programa']]

,vlr_investimento,aux_emp_pavimentacao,aux_emp_implantacao,programa
36505,1020000.0,PAVIMENTACAO,NaN,EMENDAS
36974,510000.0,PAVIMENTACAO,NaN,EMENDAS
67572,1000000.0,NaN,NaN,EMENDAS


In [32]:
1020000.0	+ 510000.0

1530000.0

In [33]:
df_empreend[(df_empreend['Código IBGE'] == codigo) & (df_empreend['programa'] == 'EMENDAS')]['vlr_investimento'].sum()

2530000.0

In [34]:
df_agrupado_municipio_empreend_anos_juntos = df_agrupado_municipio_empreend_anos_juntos.reset_index()

In [35]:
for i in df_agrupado_municipio_empreend_anos_juntos.columns:
    print(i)

Código IBGE
vlr_investimento
Populacao
pop_beneficiada
num_total_empreendimentos
aux_emp_implantacao
aux_emp_implantacao_vlr_investimento
aux_emp_pavimentacao
aux_emp_pavimentacao_vlr_investimento
aux_emp_refrota
aux_emp_refrota_vlr_investimento
aux_emp_sinalizacao
aux_emp_sinalizacao_vlr_investimento
aux_emp_calcamento
aux_emp_calcamento_vlr_investimento
aux_emp_ciclovia
aux_emp_ciclovia_vlr_investimento
aux_emp_corredor
aux_emp_corredor_vlr_investimento
aux_emp_drenagem
aux_emp_drenagem_vlr_investimento
aux_emp_ferrovia
aux_emp_ferrovia_vlr_investimento
aux_emp_transporte
aux_emp_transporte_vlr_investimento
aux_emp_brt
aux_emp_brt_vlr_investimento
aux_emp_vlt
aux_emp_vlt_vlr_investimento
aux_emp_qualificacao
aux_emp_qualificacao_vlr_investimento
aux_emp_mobilidade_urbana
aux_emp_mobilidade_urbana_vlr_investimento
aux_emp_aquisicao
aux_emp_aquisicao_vlr_investimento
aux_emp_polongamento
aux_emp_polongamento_vlr_investimento
aux_emp_ampliacao
aux_emp_ampliacao_vlr_investimento
aux_emp_

In [36]:
df_agrupado_municipio_empreend_anos_juntos.shape

(5545, 77)

In [37]:
unique_rows = df_agrupado_municipio_empreend_anos_juntos.duplicated(subset=['Código IBGE'], keep=False)

if unique_rows.sum() == 0:
    print("O conjunto dos valores da coluna 'Código IBGE' é único.")
else:
    print("O conjunto dos valores da coluna 'Código IBGE' não é único.")


O conjunto dos valores da coluna 'Código IBGE' é único.


In [38]:
df_acidentes_agrupado_ano = df_acidentes.groupby('Código IBGE').mean()

In [39]:
df_acidentes_agrupado_ano['ano'].unique()

array([2014.5])

In [40]:
df_acidentes_agrupado_ano.shape

(5570, 43)

In [41]:
df_merge_empreendimentos_acidentes_anos_juntos = pd.merge(df_agrupado_municipio_empreend_anos_juntos, df_acidentes_agrupado_ano, on=['Código IBGE'], how='outer')
df_merge_empreendimentos_acidentes_anos_juntos.shape

(5570, 120)

In [42]:
for i in df_merge_empreendimentos_acidentes_anos_juntos.columns:
    print(i)

Código IBGE
vlr_investimento
Populacao
pop_beneficiada
num_total_empreendimentos
aux_emp_implantacao
aux_emp_implantacao_vlr_investimento
aux_emp_pavimentacao
aux_emp_pavimentacao_vlr_investimento
aux_emp_refrota
aux_emp_refrota_vlr_investimento
aux_emp_sinalizacao
aux_emp_sinalizacao_vlr_investimento
aux_emp_calcamento
aux_emp_calcamento_vlr_investimento
aux_emp_ciclovia
aux_emp_ciclovia_vlr_investimento
aux_emp_corredor
aux_emp_corredor_vlr_investimento
aux_emp_drenagem
aux_emp_drenagem_vlr_investimento
aux_emp_ferrovia
aux_emp_ferrovia_vlr_investimento
aux_emp_transporte
aux_emp_transporte_vlr_investimento
aux_emp_brt
aux_emp_brt_vlr_investimento
aux_emp_vlt
aux_emp_vlt_vlr_investimento
aux_emp_qualificacao
aux_emp_qualificacao_vlr_investimento
aux_emp_mobilidade_urbana
aux_emp_mobilidade_urbana_vlr_investimento
aux_emp_aquisicao
aux_emp_aquisicao_vlr_investimento
aux_emp_polongamento
aux_emp_polongamento_vlr_investimento
aux_emp_ampliacao
aux_emp_ampliacao_vlr_investimento
aux_emp_

In [43]:
pd.merge(df_agrupado_municipio_empreend_anos_juntos, df_acidentes_agrupado_ano, on=['Código IBGE'], how='inner').shape

(5545, 120)

In [44]:
unique_rows = df_merge_empreendimentos_acidentes_anos_juntos.duplicated(subset=['Código IBGE'], keep=False)

if unique_rows.sum() == 0:
    print("O conjunto dos valores da coluna 'Código IBGE' é único.")
else:
    print("O conjunto dos valores da coluna 'Código IBGE' não é único.")


O conjunto dos valores da coluna 'Código IBGE' é único.


In [45]:
df_merge_empreendimentos_acidentes_anos_juntos

,Código IBGE,vlr_investimento,Populacao,pop_beneficiada,num_total_empreendimentos,aux_emp_implantacao,aux_emp_implantacao_vlr_investimento,aux_emp_pavimentacao,aux_emp_pavimentacao_vlr_investimento,aux_emp_refrota,aux_emp_refrota_vlr_investimento,aux_emp_sinalizacao,aux_emp_sinalizacao_vlr_investimento,aux_emp_calcamento,aux_emp_calcamento_vlr_investimento,aux_emp_ciclovia,aux_emp_ciclovia_vlr_investimento,aux_emp_corredor,aux_emp_corredor_vlr_investimento,aux_emp_drenagem,aux_emp_drenagem_vlr_investimento,aux_emp_ferrovia,aux_emp_ferrovia_vlr_investimento,aux_emp_transporte,aux_emp_transporte_vlr_investimento,aux_emp_brt,aux_emp_brt_vlr_investimento,aux_emp_vlt,aux_emp_vlt_vlr_investimento,aux_emp_qualificacao,aux_emp_qualificacao_vlr_investimento,aux_emp_mobilidade_urbana,aux_emp_mobilidade_urbana_vlr_investimento,aux_emp_aquisicao,aux_emp_aquisicao_vlr_investimento,aux_emp_polongamento,aux_emp_polongamento_vlr_investimento,aux_emp_ampliacao,aux_emp_ampliacao_vlr_investimento,aux_emp_recapeamento,aux_emp_recapeamento_vlr_investimento,aux_emp_linha,aux_emp_linha_vlr_investimento,aux_emp_metro,aux_emp_metro_vlr_investimento,aux_emp_viaria,aux_emp_viaria_vlr_investimento,aux_emp_aero,aux_emp_aero_vlr_investimento,aux_emp_revitalizacao,aux_emp_revitalizacao_vlr_investimento,aux_emp_infraestrutura,aux_emp_infraestrutura_vlr_investimento,aux_emp_pac2etapa,aux_emp_pac2etapa_vlr_investimento,aux_emp_faixa_exclusiva,aux_emp_faixa_exclusiva_vlr_investimento,aux_emp_rapido,aux_emp_rapido_vlr_investimento,aux_emp_sarjeta,aux_emp_sarjeta_vlr_investimento,aux_emp_nan,aux_emp_nan_vlr_investimento,programa_AVANÇAR PÚBLICO,programa_AVANÇAR PÚBLICO_vlr_investimento,programa_EMENDAS,programa_EMENDAS_vlr_investimento,programa_PAC FIN,programa_PAC FIN_vlr_investimento,programa_PAC OGU,programa_PAC OGU_vlr_investimento,programa_PRÓ COMUNIDADE,programa_PRÓ COMUNIDADE_vlr_investimento,programa_SETOR PRIVADO,programa_SETOR PRIVADO_vlr_investimento,vlr_investimento_per_capita,pop_beneficiada_per_capita,ano,Pedestre_mortes,Ciclista_mortes,Motociclista_mortes,Ocup_triciclo_motor_mortes,Ocup_automovel_mortes,Ocup_caminhonete_mortes,Ocup_veic_transp_pesado_mortes,Ocup_onibus_mortes,Outros_mortes,Populacao_acidentes,total_mortes,taxa_mun_mortes,taxa_mun_pedestre_mortes,taxa_mun_ciclista_mortes,taxa_mun_motociclista_mortes,taxa_mun_automovel_mortes,taxa_mun_veiculo_pesado_mortes,taxa_mun_onibus_mortes,taxa_mun_outros_mortes,Pedestre_feridos,Ciclista_feridos,Motociclista_feridos,Ocup_triciclo_motor_feridos,Ocup_automovel_feridos,Ocup_caminhonete_feridos,Ocup_veic_transp_pesado_feridos,Ocup_onibus_feridos,Outros_feridos,total_feridos,taxa_mun_feridos,taxa_mun_pedestre_feridos,taxa_mun_ciclista_feridos,taxa_mun_motociclista_feridos,taxa_mun_automovel_feridos,taxa_mun_veiculo_pesado_feridos,taxa_mun_onibus_feridos,taxa_mun_outros_feridos,Populacao_populacao,taxa_mun_triciclo_mortes,taxa_mun_caminhonete_mortes,taxa_mun_triciclo_feridos,taxa_mun_caminhonete_feridos
0,1100015.0,2530000.00,23657.333333,3034.0,3.0,0.0,0.00,2.0,1530000.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,3.0,2530000.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,3.0,2530000.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,106.943583,0.128248,2014.5,0.4,0.1,3.1,0.0,0.0,0.0,0.0,0.1,1.2,24670.2,4.9,0.019860,0.001624,0.000391,0.012685,0.000000,0.000000,0.000390,0.004770,0.0,0.3,3.7,0.0,0.1,0.0,0.0,0.0,0.2,4.3,0.017275,0.000000,0.001176,0.014921,0.000392,0.000000,0.000000,0.000785,24670.2,0.0,0.000000,0.000000,0.000000
1,1100023.0,51728180.31,80144.200000,60220.0,15.0,3.0,2938773.14,10.0,45917178.98,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1251042.3,0.0,0.0,6.0,7144333.85,0.0,0.0,1.0,36842105.26,0.0,0.0,0.0,0.0,1.0,36842105.26,5.0,40480739.47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3225839.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,14.0,148

In [46]:
df_merge_empreendimentos_acidentes_anos_juntos.to_csv('../dados/tratados/agrupamento_empreend_acidentes_por_municipio.csv', index=False)